# Experiment 4 Text Generation RNN/LSTM

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("/content/poems-100.csv")

text = " ".join(df.iloc[:,0].astype(str)).lower()
words = text.split()

vocab = list(set(words))
w2i = {w:i for i,w in enumerate(vocab)}
i2w = {i:w for w,i in w2i.items()}

data = np.array([w2i[w] for w in words])

print("Total Words:", len(words))
print("Vocabulary:", len(vocab))


Total Words: 24734
Vocabulary: 6989


In [3]:
MAX_WORDS = 5000
data = data[:MAX_WORDS]

seq = 5
X,Y = [],[]

for i in range(len(data)-seq):
    X.append(data[i:i+seq])
    Y.append(data[i+seq])

X = torch.tensor(X)
Y = torch.tensor(Y)

V = len(vocab)

print("Samples:",len(X))


Samples: 4995


/tmp/ipython-input-865247540.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  X = torch.tensor(X)


In [4]:
class OneHot(nn.Module):
    def __init__(self,cell):
        super().__init__()
        self.rnn = cell(V,32,batch_first=True)
        self.fc = nn.Linear(32,V)

    def forward(self,x):
        x = nn.functional.one_hot(x,V).float()
        o,_ = self.rnn(x)
        return self.fc(o[:,-1])


In [5]:
class Embed(nn.Module):
    def __init__(self,cell):
        super().__init__()
        self.emb = nn.Embedding(V,32)
        self.rnn = cell(32,32,batch_first=True)
        self.fc = nn.Linear(32,V)

    def forward(self,x):
        x = self.emb(x)
        o,_ = self.rnn(x)
        return self.fc(o[:,-1])


In [6]:
def train(m):
    opt = optim.Adam(m.parameters(),0.01)
    loss_fn = nn.CrossEntropyLoss()

    for e in range(3):
        opt.zero_grad()
        out = m(X)
        loss = loss_fn(out,Y)
        loss.backward()
        opt.step()
        print("Epoch",e+1,"Loss:",loss.item())


In [7]:
models = {
    "RNN OneHot": OneHot(nn.RNN),
    "LSTM OneHot": OneHot(nn.LSTM),
    "RNN Embed": Embed(nn.RNN),
    "LSTM Embed": Embed(nn.LSTM)
}

trained = {}

for k,m in models.items():
    print("\n",k)
    train(m)
    trained[k] = m



 RNN OneHot
Epoch 1 Loss: 8.861047744750977
Epoch 2 Loss: 8.728211402893066
Epoch 3 Loss: 8.555278778076172

 LSTM OneHot
Epoch 1 Loss: 8.869963645935059
Epoch 2 Loss: 8.815950393676758
Epoch 3 Loss: 8.751444816589355

 RNN Embed
Epoch 1 Loss: 8.892857551574707
Epoch 2 Loss: 8.743245124816895
Epoch 3 Loss: 8.586085319519043

 LSTM Embed
Epoch 1 Loss: 8.862014770507812
Epoch 2 Loss: 8.798221588134766
Epoch 3 Loss: 8.72358226776123


In [8]:
def generate(m,start):
    m.eval()
    w = start.split()

    for _ in range(20):
        t = torch.tensor([[w2i[i] for i in w[-seq:]]])
        nxt = m(t).argmax().item()
        w.append(i2w[nxt])

    return " ".join(w)

for k,m in trained.items():
    print("\n",k)
    print(generate(m,"the sun is"))



 RNN OneHot
the sun is crowded time time time and and and and and and and and and and and and and and and and

 LSTM OneHot
the sun is spread spread spread spread spread spread spread spread spread spread spread spread spread spread spread spread spread spread spread spread

 RNN Embed
the sun is fry. youth lived began life burn! self or tired, dreams death. marked old associates, older meat can follow glory-garland atom

 LSTM Embed
the sun is jesus’ jesus’ mean; supple use dear) pass, saved gang dinner, mean; use dear) but but melt none heaven smoke i


Observations:

1. One-Hot models consume more memory and train slower.

2. Embedding models converge faster and produce smoother sentences.

3. RNN struggles with long context.

4. LSTM captures better sequence dependencies.

Conclusion:
LSTM with Trainable Embeddings gives best performance and text quality.
